In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.ndimage import binary_fill_holes, label, generate_binary_structure, convolve
from scipy.stats import linregress

from datetime import datetime
import os
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed


In [151]:
def generate_lattice(width, height, prob):
    """
    Generate a binary lattice (2D NumPy array) where each site is True with probability `prob`.
    
    Parameters:
        width (int): Width of the lattice (number of columns).
        height (int): Height of the lattice (number of rows).
        prob (float): Probability that a site is occupied (True).
    
    Returns:
        np.ndarray of bool: Binary lattice with shape (height, width).
    """
    return np.random.rand(height, width) < prob

def filter_features_under_threshold(labeled_sky, threshold):
    """
    Remove labeled regions whose size (number of pixels) is below a threshold.
    Then relabel the remaining features contiguously.
    
    Parameters:
        labeled_sky (np.ndarray): 2D array where each connected region has a unique integer label.
        threshold (int): Minimum size of a feature to keep.
    
    Returns:
        filtered_labels (np.ndarray): 2D array of relabeled features (small ones removed).
        new_count (int): Number of features after filtering and relabeling.
    """
    counts = np.bincount(labeled_sky.ravel())  # Count pixels per label
    valid = counts > threshold                      # Mask of valid labels
    valid[0] = False                                 # Label 0 is background, always discard
    mask = valid[labeled_sky]                  # Mask to retain valid pixels
    filtered_labels, new_count = label(mask, structure=generate_binary_structure(rank=2, connectivity=1))
    return filtered_labels, new_count

def count_edge_contacts(coords, shape):
    """
    Count how many pixels from a given feature touch each lattice edge (left, top, right, bottom).
    
    Parameters:
        coords (np.ndarray): Nx2 array of (row, col) indices for a feature.
        shape (tuple): (height, width) of the full image.
    
    Returns:
        np.ndarray of int: Array of counts: [left_edge, top_edge, right_edge, bottom_edge]
    """
    rows, cols = coords[:, 0], coords[:, 1]
    height, width = shape
    return np.array([
        np.sum(cols == 0),           # Touching left edge
        np.sum(rows == 0),           # Touching top edge
        np.sum(cols == width - 1),   # Touching right edge
        np.sum(rows == height - 1)   # Touching bottom edge
    ])

def interpret_edge_touch(edge_counts):
    """
    Interpret which and how many edges a feature touches.
    
    Parameters:
        edge_counts (np.ndarray): 4-element array with counts of edge contacts.
    
    Returns:
        int:
            0 = touches no edge (fully enclosed),
            1 = touches one edge only,
           -1 = touches opposite edges (spanning),
            2 = touches two adjacent edges (corner-touch).
    """
    left, top, right, bottom = edge_counts > 0
    num_touched = np.count_nonzero(edge_counts)
    
    if num_touched == 0:
        return 0
    elif num_touched == 1:
        return 1
    elif num_touched == 2:
        if (left and right) or (top and bottom):
            return -1  # Feature spans across the image
        else:
            return 2   # Touches two adjacent edges (e.g., corner)
    else:
        return -1      # Touches 3+ edges → definitely not enclosed

def crop_to_bounding_box(mask):
    """
    Crop a binary mask to its tightest bounding box.
    
    Parameters:
        mask (np.ndarray of bool): Binary image mask.
    
    Returns:
        np.ndarray of bool: Cropped mask containing only the non-zero region.
    """
    rows, cols = np.where(mask)
    if rows.size == 0:
        return np.zeros((0, 0), dtype=bool)  # Empty mask
    return mask[rows.min():rows.max()+1, cols.min():cols.max()+1]

def slice_cloud(cloud, slice_frac):
    """
    Slice a binary cloud vertically at a given fraction of its width.
    Returns the left portion and the number of exposed pixels at the cut edge.
    
    Parameters:
        cloud (np.ndarray): 2D binary array representing the cloud.
        slice_frac (float): Fraction (0 < slice_frac < 1) of the width to retain.

    Returns:
        sliced (np.ndarray): Left-side binary array.
        exposed_cut_pixels (int): Number of True pixels in the last column of the slice.
    """
    if not (0 < slice_frac < 1):
        raise ValueError("slice_frac must be between 0 and 1 (exclusive).")

    h, w = cloud.shape
    slice_col = max(1, int(w * slice_frac))  # Ensure at least 1 column is kept

    sliced = cloud[:, :slice_col]
    exposed_cut_pixels = np.count_nonzero(sliced[:, -1])

    return sliced, exposed_cut_pixels

def slice_cloud_into_segments(cloud, num_slices):
    """
    Slice a binary cloud into non-overlapping vertical segments.
    Returns:
        - the slices,
        - the number of pixels shared across each internal slice boundary,
        - the naive right edge pixel counts.

    Parameters:
        cloud (np.ndarray): 2D binary array (cropped mask of a single cloud).
        num_slices (int): Number of vertical slices to divide the cloud into.

    Returns:
        slices (List[np.ndarray]): List of 2D binary slices that together reconstruct the cloud.
        shared_edges (List[int]): shared_edges[i] = number of shared pixels between slice i and i+1.
        naive_r_exposed (List[int]): number of nonzero pixels in the right edge of each slice.
    """
    h, w = cloud.shape
    base_width = w // num_slices
    remainder = w % num_slices

    slices = []
    naive_r_exposed = []
    left_edges = []
    right_edges = []

    start_col = 0
    for i in range(num_slices):
        slice_width = base_width + (1 if i < remainder else 0)
        end_col = start_col + slice_width

        segment = cloud[:, start_col:end_col]
        slices.append(segment)
        left_edges.append(segment[:, 0].astype(bool))
        right_edges.append(segment[:, -1].astype(bool))
        naive_r_exposed.append(np.count_nonzero(segment[:, -1]))

        start_col = end_col

    shared_edges = []
    for i in range(num_slices - 1):
        shared = np.logical_and(right_edges[i], left_edges[i + 1])
        shared_edges.append(np.count_nonzero(shared))

    return slices, shared_edges, naive_r_exposed


def compute_area(mask):
    return np.count_nonzero(mask)

def compute_perimeter(mask):
    """
    Compute the true perimeter of a binary mask by counting foreground-background edges.
    This counts how many 4-connected neighbor edges a foreground pixel has with the background.

    Parameters:
        mask (np.ndarray): 2D binary array (bool or 0/1)

    Returns:
        int: Total number of foreground-background edges
    """
    mask = mask.astype(np.uint8)

    # Define 4-connectivity kernel to count neighbors
    kernel = np.array([[0, 1, 0],
                       [1, 0, 1],
                       [0, 1, 0]], dtype=np.uint8)

    neighbor_counts = convolve(mask, kernel, mode='constant', cval=0)

    # For each foreground pixel, count how many of its 4 neighbors are background
    perimeter = np.sum(mask * (4 - neighbor_counts))

    return int(perimeter)

def compute_D(areas, perims):
    """
    Estimate the fractal dimension D via perimeter-area scaling (log-log slope).
    
    D = 2 * slope of log(perimeter) vs log(area), assuming P ~ A^{D/2}.
    
    Parameters:
        areas (array-like): List of areas of multiple features.
        perims (array-like): List of corresponding perimeters.
    
    Returns:
        float or None: Estimated fractal dimension D, or None if not enough data.
    """
    if len(areas) < 2 or len(perims) < 2:
        return None  # Not enough data for regression
    log_area = np.log(areas)
    log_perim = np.log(perims)
    slope, _, r_value, _, _ = linregress(log_area, log_perim)
    D = 2 * slope
    return D

In [152]:
def plot_lattice_readable(lattice, title=None, cmap='gray'):
    """
    Display a 2D lattice with pixel-accurate rendering and a fixed human-readable size.

    Parameters:
        lattice (np.ndarray): 2D binary or labeled array to display.
        title (str, optional): Title to show above the plot.
        cmap (str): Colormap to use. Default is 'gray'.
    """
    lattice = (lattice != 0).astype(np.uint8)
    fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
    ax.imshow(lattice, cmap=cmap, interpolation='none')
    ax.axis('off')
    if title:
        ax.set_title(title)
    plt.tight_layout()
    plt.show()

def plot_lattice_pixel_accurate(lattice):
    """
    Display a binary lattice as a pixel-accurate image (1 array element = 1 pixel).
    
    Parameters:
        lattice (np.ndarray): 2D array (e.g., bool or binary values) to display.
    """
    lattice = (lattice != 0).astype(np.uint8)
    height, width = lattice.shape
    dpi = 100  # Keep DPI fixed; actual pixel dimensions = width x height

    fig = plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(lattice, cmap='gray', interpolation='none')
    ax.axis('off')
    plt.show()

def save_image_pixel_accurate(lattice, path_prefix):
    """
    Save a 2D lattice array to disk with pixel-accurate dimensions and a timestamped filename.
    
    Parameters:
        lattice (np.ndarray): 2D array (binary or grayscale) to save as an image.
        path_prefix (str): Path and filename prefix (e.g. 'out/lattice'). Extension is auto '.png'.
    
    Returns:
        str: Full path of the saved image.
    """
    lattice = (lattice != 0).astype(np.uint8)
    height, width = lattice.shape
    dpi = 100
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S_%f')
    filename = f"{path_prefix}_{timestamp}.png"
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    fig = plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(lattice, cmap='gray', interpolation='none')
    ax.axis('off')
    fig.savefig(filename, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    return filename

def visualize_cloud_mirroring(cropped, sliced, mirrored, slice_frac, cloud_id=None):
    """
    Show cropped cloud, the sliced half, and the mirrored result.
    Optionally show the shared edge used in the correction.
    """
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    axs[0].imshow(cropped, cmap='gray')
    axs[0].set_title(f"Original Cloud\n{cropped.shape}")

    axs[1].imshow(sliced, cmap='gray')
    axs[1].set_title(f"Sliced (frac={slice_frac:.3f})\n{sliced.shape}")

    axs[2].imshow(mirrored, cmap='gray')
    axs[2].set_title(f"Mirrored Cloud\n{mirrored.shape}")

    for ax in axs:
        ax.axis('off')

    if cloud_id is not None:
        fig.suptitle(f"Cloud ID {cloud_id}", fontsize=14)

    plt.tight_layout()
    plt.show()

def visualize_shared_edge(sliced):
    """
    Show which pixels were counted as the shared edge in slicing.
    Useful for confirming shared_edge_len logic.
    """
    shared_edge_mask = np.zeros_like(sliced, dtype=np.uint8)
    shared_edge_mask[:, 0] = sliced[:, 0]  # highlight first column

    plt.figure(figsize=(4, 4))
    plt.imshow(shared_edge_mask, cmap='Reds')
    plt.title("Shared Edge Pixels (Exposed Cut Column)")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [153]:
LATTICE_SIZE = 10000
FILL_PROB = 0.405
AREA_THRESHOLD = 1000
NUM_LATTICES = 1
NUM_SLICES = 10

In [154]:
def process_lattice(index, seed, lattice_size, fill_prob, area_thresh, num_slices):
    print(f"[Lattice {index}] Running on PID {os.getpid()}")
    np.random.seed(seed)
    
    full_sky = generate_lattice(lattice_size, lattice_size, fill_prob)
    full_sky = binary_fill_holes(full_sky)
    labelled_sky, num_clouds = label(full_sky)
    labelled_sky, num_clouds = filter_features_under_threshold(labelled_sky, area_thresh)

    full_areas = []
    full_perims = []
    mirrored_areas = [[] for _ in range(num_slices)]
    mirrored_perims = [[] for _ in range(num_slices)]

    for cloud_id in tqdm(range(1, num_clouds + 1), desc=f"Lattice {index} Clouds", leave=False):
        mask = (labelled_sky == cloud_id)
        coords = np.argwhere(mask)
        if interpret_edge_touch(count_edge_contacts(coords, mask.shape)) == 0:
            single_cloud = crop_to_bounding_box(mask)
            area = compute_area(single_cloud)
            perim = compute_perimeter(single_cloud)
            full_areas.append(area)
            full_perims.append(perim)

            slices, shared_edges, r_exposed = slice_cloud_into_segments(single_cloud, num_slices)
            raw_slice_areas = [compute_area(slc) for slc in slices]
            raw_slice_perims = [compute_perimeter(slc) for slc in slices]

            for i in range(num_slices):
                mirr_area = np.sum(raw_slice_areas[:i+1]) * 2
                raw_perim = np.sum(raw_slice_perims[:i+1])
                shared = np.sum(shared_edges[:i]) if i > 0 else 0
                right_cut = r_exposed[i]
                mirr_perim = (raw_perim - 2*shared - right_cut) * 2

                mirrored_areas[i].append(mirr_area)
                mirrored_perims[i].append(mirr_perim)

    return full_perims, mirrored_perims

In [142]:
for _ in range(NUM_LATTICES):
    # populate the sky with site-percolated cells
    full_sky = generate_lattice(LATTICE_SIZE,LATTICE_SIZE, FILL_PROB)

    # fill in the clouds 
    full_sky = binary_fill_holes(full_sky)
    labelled_sky, num_clouds = label(full_sky)
    labelled_sky, num_clouds = filter_features_under_threshold(labelled_sky, AREA_THRESHOLD)

    full_areas = []
    full_perims = []
    mirrored_areas = [[] for _ in range(NUM_SLICES)]
    mirrored_perims = [[] for _ in range(NUM_SLICES)]

    for cloud_id in range(1, num_clouds + 1):
        mask = (labelled_sky == cloud_id)
        coords = np.argwhere(mask)
        if interpret_edge_touch(count_edge_contacts(coords, mask.shape)) == 0:

            single_cloud = crop_to_bounding_box(mask)

            area = compute_area(single_cloud)
            perim = compute_perimeter(single_cloud)
            full_areas.append(area)
            full_perims.append(perim)

            slices, shared_edges, r_exposed = slice_cloud_into_segments(single_cloud, NUM_SLICES)
            raw_slice_areas = []
            raw_slice_perims = []

            for i in range(NUM_SLICES):
                raw_slice_areas.append(compute_area(slices[i]))
                raw_slice_perims.append(compute_perimeter(slices[i]))

            for i in range(NUM_SLICES):
                # Area: sum areas of slices 0 through i, then mirror
                mirr_area = np.sum(raw_slice_areas[0:i+1]) * 2

                # Perimeter: sum raw perimeters for slices 0 through i
                raw_perim = np.sum(raw_slice_perims[0:i+1])

                # Subtract shared internal edges
                shared = np.sum(shared_edges[0:i]) if i > 0 else 0

                # Subtract naive right edge of the last slice
                right_cut = r_exposed[i]

                mirr_perim = (raw_perim - 2*shared - right_cut) * 2

                mirrored_areas[i].append(mirr_area)
                mirrored_perims[i].append(mirr_perim)

print(full_perims)
print(mirrored_perims)
            
                
            


[560, 584, 542, 464, 848, 656, 1524, 674, 574, 444, 832, 480, 7724, 458, 798, 1248, 516, 680, 828, 1154, 3190, 470, 2134, 712, 996, 1600, 724, 514, 686, 1100, 766, 396, 538, 512, 348, 578, 452, 962, 386, 7050, 1218, 452, 1442, 1376, 1142, 428, 1660, 560, 1006, 776, 1642, 1812, 436, 1194, 690, 534, 620, 1824, 400, 732, 1000, 690, 840, 2862, 466, 712, 630, 488, 1438, 622, 744, 452, 1104, 756, 644, 572, 824, 636, 3026, 1174, 714, 1098, 600, 452, 2346, 1326, 560, 508, 11268, 770, 604, 1128, 846, 890, 778, 632, 1054, 1652, 444, 1934, 1278, 486, 1298, 458, 506, 698, 656, 672, 1548, 1190, 692, 728, 4270, 960, 494, 8312, 922, 554, 1064, 1430, 1258, 6678, 764, 820, 5284, 1844, 684, 862, 722, 798, 494, 2912, 1574, 574, 16202, 744, 744, 644, 3768, 442, 440, 920, 466, 1134, 670, 520, 2176, 638, 2066, 634, 2678, 1320, 822, 1246, 514, 656, 874, 4110, 830, 2258, 600, 556, 520, 508, 842, 936, 860, 1432, 866, 2798, 390, 488, 452, 388, 368, 642, 980, 512, 470, 918, 530, 576, 496, 690, 578, 474, 2758, 51

In [143]:
print(len(full_areas))

4867


In [147]:
print(compute_D(areas=full_areas, perims=full_perims))
slice_Ds = []
for i in range(NUM_SLICES):
    slice_Ds.append(compute_D(areas=mirrored_areas[i],perims=mirrored_perims[i]))

print(slice_Ds)

1.3368036423849547
[np.float64(1.3533965437016389), np.float64(1.311263519576373), np.float64(1.3010039836249665), np.float64(1.3030830606424777), np.float64(1.3099840878572075), np.float64(1.3186478735240958), np.float64(1.324571850794723), np.float64(1.3293444833652592), np.float64(1.3339900735673267), np.float64(1.3390698831934758)]


In [155]:
if __name__ == "__main__":
    print(f"Main process PID: {os.getpid()}")
    print(f"Available CPUs: {multiprocessing.cpu_count()}")

    LATTICE_SIZE = 200
    FILL_PROB = 0.405
    AREA_THRESHOLD = 100
    NUM_LATTICES = 4
    NUM_SLICES = 10

    seeds = np.random.randint(0, 100000, size=NUM_LATTICES)

    all_full_perims = []
    all_mirrored_perims = [[] for _ in range(NUM_SLICES)]

    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_lattice, i, seeds[i], LATTICE_SIZE, FILL_PROB, AREA_THRESHOLD, NUM_SLICES) 
                   for i in range(NUM_LATTICES)]

        for future in tqdm(as_completed(futures), total=NUM_LATTICES, desc="Lattices Finished"):
            full_perims, mirrored_perims = future.result()
            all_full_perims.extend(full_perims)
            for i in range(NUM_SLICES):
                all_mirrored_perims[i].extend(mirrored_perims[i])

    print("Final full cloud perimeters:", all_full_perims)

Main process PID: 70996
Available CPUs: 16


Lattices Finished:   0%|          | 0/4 [00:00<?, ?it/s]0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: 

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.